In [1]:
%load_ext autoreload
%autoreload 2

In [33]:
import os
import re
import dependency_parser as dp

In [34]:
with open('../hindi.input.txt') as f:
    source_text = f.readlines()

In [35]:
source_text

['हिन्दी संवैधानिक रूप से भारत की प्रथम राजभाषा और भारत की सबसे अधिक बोली और समझी जाने वाली भाषा है। चीनी के बाद यह विश्व में सबसे अधिक बोली जाने वाली भाषा भी है।\n',
 '\n',
 'हिन्दी और इसकी बोलियाँ उत्तर एवं मध्य भारत के विविध राज्यों में बोली जाती हैं। भारत और अन्य देशों में ६० करोड़ से अधिक लोग हिन्दी बोलते, पढ़ते और लिखते हैं। फ़िजी, मॉरिशस, गयाना, सूरीनाम की अधिकतर और नेपाल की कुछ जनता हिन्दी बोलती है।\n',
 '\n',
 'हिन्दी राष्ट्रभाषा, राजभाषा, सम्पर्क भाषा, जनभाषा के सोपानों को पार कर विश्वभाषा बनने की ओर अग्रसर है। भाषा विकास क्षेत्र से जुड़े वैज्ञानिकों की भविष्यवाणी हिन्दी प्रेमियों के लिए बड़ी सन्तोषजनक है कि आने वाले समय में विश्वस्तर पर अन्तर्राष्ट्रीय महत्त्व की जो चन्द भाषाएँ होंगी उनमें हिन्दी भी प्रमुख होगी।\n',
 '\n',
 "हिन्दी शब्द का सम्बन्ध संस्कृत शब्द सिन्धु से माना जाता है। 'सिन्धु' सिन्ध नदी को कहते थे ओर उसी आधार पर उसके आस-पास की भूमि को सिन्धु कहने लगे। यह सिन्धु शब्द ईरानी में जाकर ‘हिन्दू’, हिन्दी और फिर ‘हिन्द’ हो गया। बाद में ईरानी धीरे-धीरे भारत के अधिक भा

In [120]:
input_file = 'hin_sentences.txt'
output_file = 'hin_sentences_parsed2.txt'
Subs = {
    ',':'\n,\n',
    '(': '\n(\n',
    ')': '\n)\n',
    '‘': '\n‘\n',
    '’': '\n’\n',
    '"': '\n"\n',
    "'": "\n'\n",
    "'": "\n'\n",
    '+': '\n+\n',
    '!': '\n!\n',
    
}
rep = dict((re.escape(k), v) for k, v in Subs.items())
pattern = re.compile("|".join(rep.keys()))
temporary_tokenized_normalized_filepath = 'temp.txt'
temporary_conll_filepath = 'temp.conll'
jarpath = 'malt.jar'
multi_tabs = re.compile('\t+')
with open('hindi.lemma.json') as lemmafile:
    loaded_lemma = json.load(lemmafile)

# Start doing this
with open(input_file) as f:
    source_text = f.readlines()[:1000]
tokenized = [dp.normalize(dp.tokenise(x, language='Hindi')) for x in source_text] 
flat = [x for y in tokenized for x in y if x!='\n']
new_tokens = [pattern.sub(lambda m: rep[re.escape(m.group(0))], text) for text in flat]
sentence_splits = [tokens.split('\n') for tokens in new_tokens]
flatten_sentence_splits = [x for y in sentence_splits for x in y if x]

with open(temporary_tokenized_normalized_filepath, 'w')as f:
    f.writelines((f'{x}\n' for x in flatten_sentence_splits))

for var in [tokenized, flat, new_tokens, sentence_splits, flatten_sentence_splits]:
    del var
    
lemma_input = [multi_tabs.sub('\t', tokens) for tokens in dp.get_pos_tags(temporary_tokenized_normalized_filepath)]
lemma_input = dp.lemmatise(lemma_input, loaded_lemma)
lemma_input = dp.tag2vert(lemma_input)
lemma_input = dp.modify_pos_tags(lemma_input)
lemma_input = [token.split('\t')[:3] for token in lemma_input]
dp.to_conll(lemma_input, 'temp.conll')
del lemma_input
sub = dp.run_malt_parser(temporary_conll_filepath, jarpath)
with open(output_file, 'w') as f:
    f.writelines((f'{x}\n' for x in sub))
for file in [temporary_tokenized_normalized_filepath, temporary_conll_filepath]:
    if os.path.exists(file):
        os.unlink(file)

b'-----------------------------------------------------------------------------\n                          MaltParser 1.4.1                             \n-----------------------------------------------------------------------------\n         MALT (Models and Algorithms for Language Technology) Group          \n             Vaxjo University and Uppsala University                         \n                             Sweden                                          \n-----------------------------------------------------------------------------\n\nStarted: Thu May 02 22:43:55 IST 2019\n  Transition system    : Arc-Standard\n  Parser configuration : Nivre with NORMAL root handling\n  Feature model        : temp.xml\n  Classifier           : liblinear\n.          \t      1\t      1s\t    115MB\n.          \t     10\t      2s\t    119MB\n.          \t    100\t      2s\t    163MB\n.........  \t    920\t      5s\t    382MB\nParsing time: 00:00:05 (5148 ms)\nFinished: Thu May 02 22:44:01 IST 20

In [111]:
sub = run_malt_parser(temporary_conll_filepath, jarpath)
with open(output_file, 'w') as f:
    f.writelines((f'{x}\n' for x in sub))

b'-----------------------------------------------------------------------------\n                          MaltParser 1.4.1                             \n-----------------------------------------------------------------------------\n         MALT (Models and Algorithms for Language Technology) Group          \n             Vaxjo University and Uppsala University                         \n                             Sweden                                          \n-----------------------------------------------------------------------------\n\nStarted: Thu May 02 21:52:22 IST 2019\n  Transition system    : Arc-Standard\n  Parser configuration : Nivre with NORMAL root handling\n  Feature model        : temp.xml\n  Classifier           : liblinear\n.          \t      1\t      1s\t    219MB\n.          \t     10\t      1s\t    227MB\n           \t     74\t      2s\t     80MB\nParsing time: 00:00:02 (2319 ms)\nFinished: Thu May 02 21:52:25 IST 2019\n'


In [117]:
source_text

['1\t01 Nov Radio mantra Bhavishyavani v24327न्यूज़ 60 सेकेंड’ मेंJEE Mains में पूरे नंबर हासिल कर कल्पित ने रचा इतिहास JEE Mains में पूरे नंबर हासिल कर कल्पित ने रचा इतिहास ‘फौजी हेयरकट नहीं समझे छात्र तो लिया योगी का नाम’ ALERT!\n',
 '2\t01 Oct Radio mantra Bhavishyavani v23511न्यूज़ 60 सेकेंड’ मेंJEE Mains में पूरे नंबर हासिल कर कल्पित ने रचा इतिहास JEE Mains में पूरे नंबर हासिल कर कल्पित ने रचा इतिहास ‘फौजी हेयरकट नहीं समझे छात्र तो लिया योगी का नाम’ ALERT!\n',
 '3\t01 Sept Radio mantra Bhavishyavani v22669न्यूज़ 60 सेकेंड’ मेंJEE Mains में पूरे नंबर हासिल कर कल्पित ने रचा इतिहास JEE Mains में पूरे नंबर हासिल कर कल्पित ने रचा इतिहास ‘फौजी हेयरकट नहीं समझे छात्र तो लिया योगी का नाम’ ALERT!\n',
 '4\t01 जून 2015: नवदीप ज्वैलर्स से 5 करोड़ की रंगदारी मांगी, अगले दिन फायरिंग।\n',
 '5\t01 फरवरी को तमिलनाडु में ‘सत्य मंगलम’ अदालत के सुबार्डीनेट जज डी.\n',
 '6\t02 केंद्रों पर होगा मूल्यांकन।\n',
 '7\t’ 02 जनवरी को ‘आप’ सांसद भगवंत मान ने कहा, ‘‘सुखबीर मंदबुद्धि बच्चे हैं।\n',
 '8\t03 Nov R

In [118]:
import re
al = re.compile(r'^[0-9]+\s?')
with open('hin_wikipedia_2016_300K-sentences.txt') as infile:
    with open('hin_sentences.txt', 'a+') as outfile:
        for line in infile:
            outfile.write(al.sub('', line))

In [129]:
import glob
glob.glob('./*.txt')

[]